In [16]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [14]:
train_data = pd.read_csv(r"C:\Users\ASHISH PAL\Downloads\DL + AI Exam Paper\DL + AI Exam Paper\Dataset\RNN\tweets_train.csv")
test_data = pd.read_csv(r"C:\Users\ASHISH PAL\Downloads\DL + AI Exam Paper\DL + AI Exam Paper\Dataset\RNN\tweets_test.csv")


In [19]:
train_data

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source,is_retweet,clean_text,Sentiment
0,Romy 👑,Bolton - England,"Tables turn, bridges burn, you live and learn.",2009-06-15 09:00:39,525,896,3854,False,2020-09-19 15:19:32,['TheSocialDilemma'],Twitter for Android,False,TheSocialDilemma is an eye opener isn t it ple...,Neutral
1,TLynn Peterson,"Black Canyon City, Arizona",Acquired disability ♿ after an accident. Livin...,2013-05-29 00:17:46,5045,5374,48152,False,2020-09-13 00:31:46,['TheSocialDilemma'],Twitter for Android,False,TheSocialDilemma If we don t agree on what is ...,Positive
2,Rutger Kosters,Netherlands,Cloud Solution Architect @NetApp | VCDX #209 |...,2014-10-07 09:34:10,574,251,1070,False,2020-09-09 21:10:33,['TheSocialDilemma'],Twitter for Android,False,Watching TheSocialDilemma scary to see social ...,Negative
3,IDFWU🗯🇿🇼🇿🇦,"East London, South Africa","YOU HAVE OPTIONS, YOU CAN’T JUST TAKE WHAT LIF...",2012-08-22 11:09:06,706,618,1320,False,2020-09-11 04:33:08,No hashtags,Twitter for Android,False,You check your social media before you pee in ...,Positive
4,Treebel,Unknown,No description,2010-09-25 07:25:31,9,13,55,False,2020-09-17 16:27:38,['thesocialdilemma'],Twitter for iPhone,False,watch thesocialdilemma and see what s actually...,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,Ra'Chelle Rogers,Philadelphia/New York,mom. media. art. Publicist. Philly. Opinions e...,2009-05-22 13:34:14,387,400,1274,False,2020-09-13 22:12:29,['TheSocialDilemma'],Twitter for iPhone,False,Watching TheSocialDilemma,Neutral
16996,Mike Lynch,"Boston, MA, USA",higher education professional . educator . doc...,2009-04-27 01:16:09,4444,4995,52692,False,2020-09-20 15:17:14,['TheSocialDilemma'],Twitter for iPhone,False,If you re not paying for the product you are t...,Neutral
16997,James Baldwin was a genius.,"Terra-Belle, USA",Class based policies won't fix the racial weal...,2012-10-05 23:27:21,879,1456,50103,False,2020-09-19 14:04:32,['TheSocialDilemma'],Twitter Web App,False,Watching TheSocialDilemma Let s see,Neutral
16998,Johan - SocialMediaBreakup,"Dublin City, Ireland",Helping people to beat their compulsive phone ...,2018-10-25 22:21:17,1009,1283,19290,False,2020-09-18 15:37:00,['TheSocialDilemma'],TweetDeck,False,Yes TheSocialDilemma what a perfect time to be...,Positive


In [29]:
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\ASHISH
[nltk_data]     PAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to C:\Users\ASHISH
[nltk_data]     PAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
def preprocess_text(text):
    """
    Preprocess text by lowercasing, removing URLs, special characters,
    and stopwords.
    """
    if not isinstance(text, str):  # Check if input is not a string
        return ''
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenize and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    
    # Rejoin the tokens
    cleaned_text = ' '.join(tokens)
    return cleaned_text


In [32]:
train_data['clean_text'] = train_data['clean_text'].apply(preprocess_text)
test_data['clean_text'] = test_data['clean_text'].apply(preprocess_text)

C:\Users\ASHISH PAL\AppData\Local\Temp\ipykernel_23288\4280027138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['clean_text'] = train_data['clean_text'].apply(preprocess_text)


In [33]:
# Step 5: Encode Sentiments
label_encoder = LabelEncoder()
train_data['Sentiment'] = label_encoder.fit_transform(train_data['Sentiment'])

C:\Users\ASHISH PAL\AppData\Local\Temp\ipykernel_23288\2723459011.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Sentiment'] = label_encoder.fit_transform(train_data['Sentiment'])


In [34]:
# Tokenize text data
tokenizer = Tokenizer(num_words=10000)  # Use top 10,000 words
tokenizer.fit_on_texts(train_data['clean_text'])

In [35]:
X_train = tokenizer.texts_to_sequences(train_data['clean_text'])
X_test = tokenizer.texts_to_sequences(test_data['clean_text'])

In [36]:
max_len = 50  # Adjust based on the dataset
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [37]:
# One-hot encode target variable
y_train = pd.get_dummies(train_data['Sentiment']).values

# Step 7: Train-Test Split for Validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [38]:
# Step 8: Build the RNN Model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_len),
    SimpleRNN(128, return_sequences=False),  # RNN layer
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')  # Output layer for 3 classes
])

C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [39]:
# Step 9: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)


Epoch 1/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.6367 - loss: 0.8191 - val_accuracy: 0.8347 - val_loss: 0.4508
Epoch 2/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.9115 - loss: 0.2654 - val_accuracy: 0.8388 - val_loss: 0.4606
Epoch 3/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9633 - loss: 0.1107 - val_accuracy: 0.8358 - val_loss: 0.5906
Epoch 4/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9821 - loss: 0.0564 - val_accuracy: 0.8282 - val_loss: 0.6735
Epoch 5/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9885 - loss: 0.0396 - val_accuracy: 0.8205 - val_loss: 0.7565
Epoch 6/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.9908 - loss: 0.0322 - val_accuracy: 0.8288 - val_loss: 0.8213
Epoch 7/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9937 - loss: 0.0218 - val_accuracy: 0.8299 - val_loss: 0.8697
Epoch 8/10
425/425 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.9919 - loss: 0.0267 - val_a

In [41]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1) 

96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [42]:
test_data['Predicted_Sentiment'] = label_encoder.inverse_transform(y_pred_labels)


In [43]:
test_data[['clean_text', 'Predicted_Sentiment']].to_csv("Test_predictions.csv", index=False)


In [44]:
print("Predictions saved to 'Test_predictions.csv'.")


Predictions saved to 'Test_predictions.csv'.


In [45]:

# Step: Decode predictions back to original sentiment labels
test_data['Predicted_Sentiment'] = label_encoder.inverse_transform(y_pred_labels)

# Step: Save the updated test file with predictions
test_data.to_csv("Test_data_with_predictions.csv", index=False)


Test file with predictions saved to 'Test_data_with_predictions.csv'.
